In [ ]:
# setting path to the directory with the features
import sys
sys.path.append(sys.path[0]+'/./Features')

# UNIVERSAL IMPORT
from universal_import import *

# import other features
import helper
import strategies_helper
import portfolio_class

# read the database
nasdaq_df = helper.get_dataframe()

In [ ]:
def monkey_yearly(investment_year, divestment_year, dataframe):
    """
    DESCRIPTION:
        Calculate the monthly ROIs of a portfolio that invests in 10 randomly selected companies from the 1000 companies with the
        largest market cap yearly.
    
    INPUT SIGNATURE:
        1. investment_year (int)
        2. divestment_year (int)
        3. dataframe (Pandas dataframe): the source of our data, usually nasdaq_df

    OUTPUT SIGNATURE:
        1. yearly_ROIs_time (dictionary): { "year-month" : [portfolio value, ROI (since the start of the portfolio)] }
            (first on the list, but will be returned last, index to this by using [-1])
        2. final_value (float): one of the final attribute of the porfolio on the divestment date
        3. final_liquid (float): like 2
        4. final_invested (float): like 2
        5. final_divested (float): like 2
        6. final_ROI (float percentage): like 2
    """

    # take into account for earliest and latest available data
    latest_month = 12

    if divestment_year == 2022:
        latest_month = 4

    # output dictionary
    yearly_ROIs = {}

    current_year = investment_year

    # list of all holdings within the portfolio updated yearly
    dynamic_holdings = []

    # create the monkey portfolio object
    monkey_portfolio = portfolio_class.portfolio()

    while current_year != divestment_year:

        # get the monthly most valuable 500 companies
        yearly_largest_1000 = strategies_helper.theXXX(1000, [current_year, 1], dataframe)

        # randomly select 10 companies
        monkey_10 = []
        for i in range (10):
            chosen = random.choice(yearly_largest_1000)
            monkey_10.append(chosen)
            yearly_largest_1000.remove(chosen)

        # buy new companies that are randomly selected by monkey
        for firm in monkey_10:

            monkey_portfolio.transaction(firm, "BOUGHT", [current_year, 1], "Close")
            dynamic_holdings.append(firm) # here, we actually wants to append a firm multiple times in case it fall off the top 500
            # then we will need to sell mulitple shares of said firm

        # get yearly ROI
        yearly_attributes = monkey_portfolio.get_portfolio_value(current_year, 12)
        this_year_ROI = yearly_attributes[-1]
        yearly_ROIs[current_year] = this_year_ROI

        current_year += 1

    # when current_year == divestment_year, we hypothetically sell all holdings to check the ROI of the portfolio
    last_year_attributes = monkey_portfolio.get_portfolio_value(divestment_year, latest_month)
    last_year_ROI = last_year_attributes[-1]

    # add the latest year to the dictionary as well
    yearly_ROIs[divestment_year] = last_year_ROI

    # final attributes to return
    final_value = last_year_attributes[0]
    final_liquid = last_year_attributes[1]
    final_invested = last_year_attributes[2]
    final_divested = last_year_attributes[3]
    final_ROI = last_year_ROI

    return final_value, final_liquid, final_invested, final_divested, final_ROI, yearly_ROIs

In [ ]:
output = monkey_yearly(2012, 2022, nasdaq_df)

yearly_ROI = output[-1]

years = list(yearly_ROI.keys())
years.sort()

corresponding_ROIs = []

# create a dataframe from the yearly_ROI dictionary
for key in years:

    ROI = yearly_ROI[key]
    corresponding_ROIs.append(ROI)

data = {"Year":years, "ROI":corresponding_ROIs}

yearly_ROI_df = pd.DataFrame(data)

# export the data
yearly_ROI_df.to_csv("Internal Data/monkey_yearly_ROI.csv", index = False)

In [ ]:
output

In [ ]:
print("Portfolio's Performance if a monkey randomly chooses 10 stock year year to invest in\
    \n\nPortfolio's Value Upon Immediate Liquidation:\n$", output[0],\
    "\n\nCash Flow (from $0):\n$", output[1],\
    "\n\nTotal Amount Invested:\n$", output[2],\
    "\n\nTotal Amount Divested:\n$", output[3],\
    "\n\nReturn on Investment Since the Initialization of the Portfolio:\n%", output[4],\
    "\n\nDetailed yearly ROI (%):\n", output[5])